# Objetivo do Notebook
Esse notebook tem como objetivo dividir a base de dados das coletas feitas pelo Instituto de Geociências da Unicamp (IG) com base na data em que as coletas foram feitas e aplicar o algoritmo *Self Organizing Maps* (SOM) para obter grupos de poços relativamente semelhantes com base nos dados fornecidos. Para isso, é necessário que a base de dados das coletas esteja tratada.

In [1]:
#-----------------------------------------------------#
#              IMPORTAÇÃO DAS BIBIOTECAS              #
#-----------------------------------------------------#

!pip install pyproj
!pip install folium
!pip install sklearn-som
!pip install git+https://github.com/IanAguiar-ai/metrics.git
!pip install git+https://github.com/IanAguiar-ai/Self_Organizing_Maps.git

from pyproj import Proj
import folium
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import plotly.express as px
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from random import seed, random
from mutual_information import metrics as mtr
import som.som as som
from sklearn_som.som import SOM
import itertools

  Cloning https://github.com/IanAguiar-ai/metrics.git to /tmp/pip-req-build-59u4x89t
  Running command git clone --filter=blob:none --quiet https://github.com/IanAguiar-ai/metrics.git /tmp/pip-req-build-59u4x89t
  Resolved https://github.com/IanAguiar-ai/metrics.git to commit c7b50f89d88ec2fbec32bdc7f5963ee5b8586e92
  Preparing metadata (setup.py) ... done
  Created wheel for metrics: filename=metrics-0.0.11-py3-none-any.whl size=17649 sha256=631996d4b1c0f0d77a06b7bbb12387e4223875ed0c6c81e19b86a917dc5eef72
  Stored in directory: /tmp/pip-ephem-wheel-cache-2xfdcldr/wheels/c7/48/53/4f50b27b511fdefedb30381ad08615e2dc2bae6ab5a0986841
Successfully built metrics
  Cloning https://github.com/IanAguiar-ai/Self_Organizing_Maps.git to /tmp/pip-req-build-tefzuf0a
  Running command git clone --filter=blob:none --quiet https://github.com/IanAguiar-ai/Self_Organizing_Maps.git /tmp/pip-req-build-tefzuf0a
  Resolved https://github.com/IanAguiar-ai/Self_Organizing_Maps.git to commit 3d5d474cadc8817e3f0

# Importação dos arquivos
Para esse notebook, será necessário apenas o arquivo da base de dados das coletas, de forma já tratada, em arquivo `.csv`.

In [2]:
#-----------------------------------------------------#
#               IMPORTAÇÃO DOS ARQUIVOS               #
#-----------------------------------------------------#

from google.colab import drive
drive.mount('/content/drive')

# df = Base de dados
df = pd.read_csv('/content/drive/Shareddrives/datasci4water/IG/data/interim/df_final.csv')
df = df.reset_index(drop=True)

Mounted at /content/drive


# Separação da base de dados principal de acordo com a data:
Com a base de dados principal, será criado duas outras novas base de dados. Uma para cada período de coleta. Portanto a data é a variável responsável por essa divisão. Assim, formam-se, bases de dados para os seguintes períodos:


*   Abril de 2019
*   Outubro de 2019



In [3]:
#-----------------------------------------------------#
#         SEPARAÇÃO DA BASE DE DADOS POR DATA         #
#-----------------------------------------------------#

# Fazer a separação por data
df_abril = df.loc[df['date'] == '2019/04']
df_outubro = df.loc[df['date'] == '2019/10']

# Padronização dos valores
Como a base de dados possuem valores não numéricos referentes ao poço da amostra (coluna "*well*") e referente a data da coleta (coluna "*date*"), faremos a separação apenas dos valores numéricos para que o trabalho do algoritmo de aprendizado de máquina seja apenas sobre esses valores numéricos.

Ademais, como os valores possuem uma grande diferença entre ordens de grandeza, será feito a padronização desses valores, mantendo-os na mesma escala. Dessa  forma, pode-se evitar que variáveis com altas ordens de grandeza implique no cálculo de grandes distâncias entre os dados e influencie negativamente o funcionamento do algoritmo *K-Means*.

In [4]:
#-----------------------------------------------------#
#               PADRONIZAÇÃO DOS VALORES              #
#-----------------------------------------------------#

#  Separação dos valores numéricos
df_abril_values = df_abril.iloc[:, 2:df_abril.shape[1]]
df_outubro_values = df_outubro.iloc[:, 2:df_outubro.shape[1]]

# Padronização dos vallores propriamente dito
df_scaler = StandardScaler()
df_abril_values = df_scaler.fit_transform(df_abril_values)
df_outubro_values = df_scaler.fit_transform(df_outubro_values)

# Mudança no sistema de coordenadas
As coordenadas no Dataframe estão no sistema UTM. Para a utilização da biblioteca *folium* no qual serão impressos os rótulos do agrupamento, as coordenadas devem estar no sistema de graus decimais. Para isso, faz-se a conversão dos valores


In [5]:
#-----------------------------------------------------#
#                MUDANÇA DE COORDENADAS               #
#-----------------------------------------------------#

def utm_para_graus_decimais(easting, northing, lista_coordenadas):
    zona = 23
    hemisferio = 'S'
    proj = Proj(proj='utm', zone=zona, south=True, ellps='WGS84')
    longitude, latitude = proj(easting, northing, inverse=True)
    lista_coordenadas.append([latitude, longitude])


coordenadas_abril = []
coordenadas_outubro = []

for i in range(df_abril.shape[0]):
  utm_n = df_abril.iloc[i, 48]
  utm_e = df_abril.iloc[i, 47]
  utm_para_graus_decimais(utm_e, utm_n, coordenadas_abril)

for i in range(df_outubro.shape[0]):
  utm_n = df_outubro.iloc[i, 48]
  utm_e = df_outubro.iloc[i, 47]
  utm_para_graus_decimais(utm_e, utm_n, coordenadas_outubro)

# Métricas para o agrupamento do algoritmo SOM
Antes de executarmos as métricas, foi necessário resolver corrigir o seguinte problema:

  *Devido ao baixo número de amostras e ao grande número de variáveis na base de dados, cada vez em que se executava o algoritmo SOM era possível fazer os grupos diferentes do experimento anterior.*

  Para contornar esse problema, para cada matriz m por n de neurônios do algoritmo, são feitos vários experimentos e o agrupamento mais frequente é considerado o melhor agrupamento para aquele matriz de neurônios. Dessa forma, fa-se uso da seguinte função.

In [6]:
def melhor_agrupamento(dados, n_execucoes, i, j):
  resultados = []
  for execucao in range(n_execucoes):
    # Instanciando e ajustando o SOM aos dados
    som = SOM(m=i, n=j, dim=dados.shape[1])
    som.fit(dados)
    resultados.append(som.predict(dados))

  # Contar a frequência de cada conjunto de grupos
  frequencias = {}
  for grupos in resultados:
      grupos_str = ' '.join(map(str, grupos))
      if grupos_str in frequencias:
          frequencias[grupos_str] += 1
      else:
          frequencias[grupos_str] = 1
  # Encontrar os grupos mais frequentes
  grupos_mais_frequentes = max(frequencias, key=frequencias.get)
  grupos_mais_frequentes = list(map(int, grupos_mais_frequentes.split()))
  print(f"Matriz SOM {i}x{j}")
  print(grupos_mais_frequentes)
  print("Frequência:", max(frequencias.values()), "| Número de grupos:", len(set(grupos_mais_frequentes)))
  return grupos_mais_frequentes

In [ ]:
clusters_list_abril = []
for i in range(2, 7):
  for j in range(2, 7):
    clusters_list_abril.append(melhor_agrupamento(df_abril_values, 20000, i, j))
clusters_list_abril = np.array(clusters_list_abril)

np.save('/content/drive/Shareddrives/datasci4water/IG/data/interim/clusters/som_clusters_list_abril.npy', clusters_list_abril)

Matriz SOM 2x2
[1, 1, 0, 3, 0, 3, 1, 3, 3, 1, 1, 0, 0, 3, 1, 3, 0, 2, 2]
Frequência: 60 | Número de grupos: 4
Matriz SOM 2x3
[5, 4, 1, 0, 2, 3, 4, 3, 3, 4, 5, 1, 1, 3, 5, 3, 0, 2, 2]
Frequência: 25 | Número de grupos: 6
Matriz SOM 2x4
[7, 7, 6, 2, 1, 3, 7, 3, 3, 7, 7, 6, 6, 3, 7, 3, 1, 5, 5]
Frequência: 7 | Número de grupos: 6
Matriz SOM 2x5
[9, 9, 8, 3, 2, 4, 9, 4, 4, 9, 9, 8, 8, 4, 9, 4, 3, 7, 7]
Frequência: 4 | Número de grupos: 6
Matriz SOM 2x6
[11, 4, 3, 5, 9, 5, 4, 5, 5, 11, 11, 3, 3, 5, 11, 5, 2, 10, 10]
Frequência: 3 | Número de grupos: 7
Matriz SOM 3x2
[5, 3, 2, 0, 4, 1, 3, 1, 1, 3, 5, 2, 2, 1, 5, 1, 0, 4, 4]
Frequência: 27 | Número de grupos: 6
Matriz SOM 3x3
[6, 3, 4, 1, 7, 0, 3, 0, 0, 3, 6, 4, 4, 0, 6, 0, 2, 7, 7]
Frequência: 3 | Número de grupos: 7
Matriz SOM 3x4
[4, 4, 8, 1, 5, 0, 4, 0, 0, 4, 4, 8, 8, 0, 4, 0, 6, 9, 9]
Frequência: 2 | Número de grupos: 7
Matriz SOM 3x5
[2, 1, 6, 5, 11, 0, 1, 0, 0, 1, 2, 6, 6, 0, 2, 0, 10, 7, 12]
Frequência: 2 | Número de grupos: 9
Matriz 

In [ ]:
clusters_list_outubro = []
for i in range(2, 7):
  for j in range(2, 7):
    clusters_list_outubro.append(melhor_agrupamento(df_outubro_values, 20000, i, j))
clusters_list_outubro = np.array(clusters_list_outubro)

np.save('/content/drive/Shareddrives/datasci4water/IG/data/interim/clusters/som_clusters_list_outubro.npy', clusters_list_outubro)

In [ ]:
clusters_list_abril = np.load('/content/drive/Shareddrives/datasci4water/IG/data/interim/clusters/som_clusters_list_abril.npy')
clusters_list_outubro = np.load('/content/drive/Shareddrives/datasci4water/IG/data/interim/clusters/som_clusters_list_outubro.npy')

In [ ]:
def combinacoes_de_matrizes(matriz):
  dict_numero_de_clusters = {}
  for i in range(matriz.shape[0]):
    grupo_str = ' '.join(map(str, matriz[i]))
    dict_numero_de_clusters[grupo_str] = len(np.unique(matriz[i]))

  quantidade_itens_impressos = 0

  # Loop para imprimir os três primeiros itens
  # for chave, valor in dict_numero_de_clusters.items():
  #     print("Agrupamento:", list(map(int, chave.split())), "| Número de grupos:", valor)
  #     quantidade_itens_impressos += 1
  #     if quantidade_itens_impressos == 3:
  #         break

  tamanho_combinacoes = len(np.unique(list(dict_numero_de_clusters.values())))
  combinacoes = list(itertools.combinations(dict_numero_de_clusters.items(), tamanho_combinacoes))

  # Exibindo apenas as combinações que têm valores diferentes
  lista_todas_combinacoes = []
  for comb in combinacoes:
    valores = [valor for chave, valor in comb]
    if len(set(valores)) == len(valores):
      dicionario_combinado = dict(comb)
      dicionario_combinado = list(dicionario_combinado.keys())

      for i in range(tamanho_combinacoes):
        dicionario_combinado[i] = dicionario_combinado[i].split()
        for j in range(matriz.shape[1]):
          dicionario_combinado[i][j] = int(dicionario_combinado[i][j])
      lista_todas_combinacoes.append(dicionario_combinado)

  return (lista_todas_combinacoes)

In [ ]:
combinacoes_abril = combinacoes_de_matrizes(clusters_list_abril)
combinacoes_abril = np.array(combinacoes_abril)

In [ ]:
combinacoes_abril[0].shape, df_abril_values.shape

In [ ]:
# todas_metricas_abril = []
# for i in range(combinacoes_abril.shape[0]):
  # todas_metricas_abril.append(mtr.print_metrics(combinacoes_abril[0], df_abril_values))
mtr.print_metrics(combinacoes_abril[0], df_abril_values)